In [1]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("dark")
plt.rcParams['figure.figsize'] = 16, 12
import pandas as pd
from tqdm import tqdm_notebook
import io
from PIL import Image
from glob import glob
from collections import defaultdict
import os
import pickle
from io import BytesIO

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder

from kaggle_camera_model_id_lib.utils import ImageList, NpzFolder, NCrops, TifFolder, TifFolderExFiles
from kaggle_camera_model_id_lib.models import VggHead, StyleVggHead

In [23]:
val_path = '/home/mephistopheies/storage2/data/camera-model-id/val/'
test_path = '/home/mephistopheies/storage2/data/camera-model-id/raw/test/'
train_path = '/home/mephistopheies/storage2/data/camera-model-id/raw/train/'
model_path = '/home/mephistopheies/storage2/data/camera-model-id/tmp_zotac/step1/best_model.tar'
out_dir = '/home/mephistopheies/storage2/data/camera-model-id/submit/'
model_type = 'Vgg19Head_E_3b_bn'
n_classes = 9
crop_size = 256
step = 128
batch_size = 5
num_workers = 1

to_tensor = transforms.ToTensor()
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

crop_center_size = 1024

random_crop = transforms.RandomCrop(crop_size)
center_crop = transforms.CenterCrop(crop_center_size)
rvf = transforms.RandomVerticalFlip()
rhf = transforms.RandomHorizontalFlip()
random_flip = lambda img: rvf(rhf(img))

def aug_train(img):
    if min(img.size) > crop_center_size:
        return random_flip(random_crop(center_crop(img)))
    return random_flip(random_crop(img))


def jpg_compress_np(x, quality=(70, 90)):
    x = Image.fromarray(x)
    out = BytesIO()
    x.save(out, format='jpeg', 
           quality=np.random.randint(quality[0], quality[1]))
    x = Image.open(out)
    return np.array(x)

def jpg_compress_pil(x, quality=(70, 90)):
    out = BytesIO()
    x.save(out, format='jpeg', 
           quality=np.random.randint(quality[0], quality[1]))
    x = Image.open(out)
    return np.array(x)

In [3]:
model_factory = {
    'Vgg19Head_E_2b_bn': lambda n_classes: VggHead(num_classes=n_classes, load_vgg_bn=True, batch_norm=True),
    'Vgg19Head_E_3b_bn': lambda n_classes: VggHead(num_classes=n_classes, load_vgg_bn=True, batch_norm=True),
    'Vgg19Head_E_bn': lambda n_classes: VggHead(num_classes=n_classes, load_vgg_bn=True, vgg_key='E', batch_norm=True),
    'Vgg11Head_A_bn': lambda n_classes: VggHead(num_classes=n_classes, load_vgg_bn=True, vgg_key='A', batch_norm=True),
    'Vgg11Head_A': lambda n_classes: VggHead(num_classes=n_classes, load_vgg_bn=True, vgg_key='A', batch_norm=False),
    'StyleVggHead_bn': lambda n_classes: StyleVggHead(num_classes=n_classes, load_vgg_bn=True),
    'IEEEfcn': lambda n_classes: IEEEfcn(n_classes)
}

model = model_factory[model_type](n_classes)
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model'])
loss_train = checkpoint['loss_train']
acc_train = checkpoint['acc_train']
loss_val = checkpoint['loss_val']
acc_val = checkpoint['acc_val']
class_to_idx = checkpoint['class_to_idx']
idx2class = dict([(v, k) for (k, v) in class_to_idx.items()])
print('Last state:\n  TLoss: %0.6f\n  TAcc:  %0.4f\n  VLoss: %0.6f\n  VAcc:  %0.4f' % 
    (loss_train[-1], acc_train[-1], loss_val[-1], acc_val[-1]))
del(checkpoint)
model = model.cuda()
model = model.eval()

Last state:
  TLoss: 0.078632
  TAcc:  0.9802
  VLoss: 0.065313
  VAcc:  0.9828


In [56]:
ds_train = ImageFolder(
    train_path,
    transform=transforms.Compose([
        transforms.Lambda(lambda img: NCrops(np.array(center_crop(img)), crop_size=crop_size, step=step)),
        transforms.Lambda(lambda crops: torch.stack([normalize(to_tensor(crop)) for crop in crops]))
    ]))
idx2class = dict([(v, k) for (k, v) in ds_train.class_to_idx.items()])
batch_size = 1
train_loader = torch.utils.data.DataLoader(    
    ds_train,
    batch_size=batch_size, 
    shuffle=False,
    num_workers=1, 
    pin_memory=True)

db = defaultdict(list)
for ix_batch, (X, Y) in tqdm_notebook(enumerate(train_loader), total=int(len(ds_train.imgs)/batch_size)):
    feats = model.vgg(Variable(X.squeeze().cuda(), volatile=True))
    db[idx2class[Y[0]]].append(feats.view(feats.shape[0], feats.shape[1], -1).mean(dim=2).cpu().data.numpy())

In [58]:
db.keys()

dict_keys(['LG-Nexus-5x', 'Samsung-Galaxy-Note3', 'iPhone-6', 'Motorola-X', 'Sony-NEX-7', 'Motorola-Droid-Maxx', 'iPhone-4s', 'Motorola-Nexus-6', 'HTC-1-M7', 'Samsung-Galaxy-S4'])